In [295]:
from datasets import load_from_disk
import pandas as pd
import re
import ast
import pyperclip
import pprint
import os
from together import Together
from openai import OpenAI
import pandas as pd 
import glob

together_client = Together(api_key=open('/Users/spangher/.togetherai-usc-key.txt').read().strip())
os.environ['OPENAI_API_KEY']=open('/Users/spangher/.openai-isi-project-key.txt').read().strip()
openai_client = OpenAI()

def query_together(prompt, client=together_client):
    response = client.chat.completions.create(
        model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
        messages=[{
                    "role": "system",
                    "content": "You are an experienced journalist."
                },
                {
                    "role": "user",
                    "content": prompt
                }],
        max_tokens=2048,
        temperature=0.1,
        top_p=0.7,
        top_k=50,
        repetition_penalty=1,
        stop=["<|eot_id|>","<|eom_id|>"],
    )
    return response.choices[0].message.content

In [1]:
! mkdir ../data
! wget https://storage.googleapis.com/usc-data/all-coref-resolved.tar.gz
! wget https://storage.googleapis.com/usc-data/full-source-scored-data.jsonl.gz
! mv all-coref-resolved.tar.gz ../data
! mv full-source-scored-data.jsonl.gz ../data
! tar -xvzf ../data/all-coref-resolved.tar.gz

mkdir: ../data: File exists
--2024-10-06 22:52:23--  https://storage.googleapis.com/usc-data/all-coref-resolved.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.189.27, 142.250.68.123, 142.250.68.91, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.189.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6256966794 (5.8G) [application/x-tar]
Saving to: ‘all-coref-resolved.tar.gz’

all-coref-resolved. 100%[===================>]   5.83G  22.8MB/s    in 5m 37s  

2024-10-06 22:58:00 (17.7 MB/s) - ‘all-coref-resolved.tar.gz’ saved [6256966794/6256966794]

--2024-10-06 22:58:00--  https://storage.googleapis.com/usc-data/full-source-scored-data.jsonl.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.72.187, 142.250.188.251, 172.217.14.91, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.72.187|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1355

In [40]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import json
data_dir = '../data'
data_dir = '../../../bloomberg-research/press-releases/data/s_p_500_backlinks'

In [35]:
test_set_df = pd.DataFrame(json.load(open('../data/v2_queries/combined_test_prompt1_v2__with_oracle.json')))

In [42]:
batches = pd.read_json(f'{data_dir}/full-source-scored-data.jsonl', lines=True, chunksize=10_000)

In [50]:
all_source_df_chunks = []
for b in tqdm(batches):
    b_filt = b.loc[lambda df: df['article_url'].isin(test_set_df['url'])]
    all_source_df_chunks.append(b_filt)

0it [00:00, ?it/s]

In [51]:
source_df = pd.concat(all_source_df_chunks)

In [370]:
article_d

Dataset({
    features: ['article_url', 'target_timestamp_key', 'target_timestamp', 'sort_criteria', 'wayback_url', 'wayback_timestamp', 'method', 'links', 'article_text', 'word_lists', 'sent_lists', 'best_class', 'coref_resolved_sents'],
    num_rows: 496380
})

In [52]:
article_d = load_from_disk(data_dir + '/all-coref-resolved')
filtered_article_d = article_d.filter(lambda x: x['article_url'] in set(source_df['article_url']), num_proc=10)

Loading dataset from disk:   0%|          | 0/41 [00:00<?, ?it/s]

Filter (num_proc=10):   0%|          | 0/496380 [00:00<?, ? examples/s]

In [53]:
#experimentation, please ignore
print(type(filtered_article_d))
articles = filtered_article_d.to_pandas()


sentences_with_quotes = (
    filtered_article_d
         .to_pandas()
         .merge(source_df, on='article_url'))

<class 'datasets.arrow_dataset.Dataset'>


In [359]:
sentences_with_quotes.shape

(2770, 18)

In [68]:
filtered_article_d

Dataset({
    features: ['article_url', 'target_timestamp_key', 'target_timestamp', 'sort_criteria', 'wayback_url', 'wayback_timestamp', 'method', 'links', 'article_text', 'word_lists', 'sent_lists', 'best_class', 'coref_resolved_sents'],
    num_rows: 2770
})

In [69]:
disallowed_quote_types = set(['Other', 'Background/Narrative', 'No Quote'])

sentences_with_quotes = (
    filtered_article_d
         .to_pandas()
         .merge(source_df, on='article_url')
)

# sentences_with_quotes = (sentences_with_quotes
#      .assign(attributions=lambda df:
#              df.apply(lambda x: x['attributions'] if x['quote_type'] not in disallowed_quote_types else np.nan, axis=1)
#     )
# )

In [224]:
def robust_extract_json_str(lm_string):
    try:
        return json.loads(lm_string)
    except:
        try:
            return ast.literal_eval(lm_string)
        except:
            pass

In [322]:
one_url = sentences_with_quotes['article_url'].unique()[0]

idx = 210
one_url = test_set_df['url'].iloc[idx]

one_article = (
    sentences_with_quotes
        .loc[lambda df: df['article_url'] == one_url]
        .reset_index(drop=True)
)

In [357]:
one_article[['article_url', 'article_text']]

,article_url,article_text
0,in.mashable.com/apps-and-software/55842/thread...,Skip to main content \n Tech \nThreads Needs ...


In [323]:
our_previously_annotated_df = (one_article[['article_url', 'attributions', 'quote_type', 'sent_lists', ]]
        .explode(['attributions', 'quote_type', 'sent_lists'])
)

In [324]:
CLEAN_ARTICLE_TEXT_PROMPT = """
You are a helpful editor assistant. I scraped this news article from the web and removed HTML tags, however, 
there is a lot of extraneous text from the HTML page that remains. Please only extract text related to the news article.
Try not to miss any text. Copy over the text exactly. Here is the news article:

```{article_text}```

Return only the text.
"""

In [325]:
prompt = CLEAN_ARTICLE_TEXT_PROMPT.format(article_text=one_article['article_text'][0])
cleaned_article_text = query_together(prompt)

In [326]:
print(cleaned_article_text)

Threads Needs User-Generated Alt Text ASAP

Of all the missing features, this one is standing out.

by Meera Navlakha 
July 6, 2023

Threads, Meta's Twitter alternative, is here, carrying some Instagram features in tow but really lacking when it comes to others. One of these is alternative text (alt text), a crucial tool for accessibility that tech platforms have been integrating for a while. So far, Threads includes "core accessibility features" that Instagram already has -- it was built by the same team. This means screen-reader support and AI-generated image descriptions are enabled on the new app. But Threads doesn't allow users to add their own text to images, and AI doesn't always do the trick.

Indiana University defines accessibility as "the degree to which a product, device, service, environment, or facility is usable by as many people as possible". In line with this, alt-text descriptions are meant to provide meaning and explanation to content, concisely explaining what each 

In [366]:
SOURCE_EXTRACTION_PROMPT = """
You are a helpful news assistant. Here is a news article:

```{news_article}```

Please summarize each informational source providing information in the article. 
Include unnamed or passively expressed sources (e.g. "witnesses", "price signals") if there is information attributable to them.
Include any facts that might have come from the source.
Make sure each source you return refer to just one source. For example: if "John and Jane" both contribute the same information, generate two separate summaries, one for "John" and one for "Jane". 
Generate only ONE summary per source.

For each source, provide the following information:
    (1) Name: just the name of the source.
    (2) Biography: A brief biography of the source mentioned in the article.
    (3) Information: Restate the facts provided by the source. Be as SPECIFIC and be as VERBOSE as possible. 
        Contextualize ALL the information the source describes. State the full names of all people, places, events and ideas mentioned and
        everything the source says with AS MUCH BACKGROUND INFORMATION from the article so I can fully understand the information
        the source is giving. I will look at each source independently without looking at any others, so help me understand the context.

Here are some examples:
example 1:
{{ "Name": "Supermarkets around the country",
   "Biography": "Retail stores that sell food and other household items",
   "Information": "Supermarkets around the country alerted shoppers that prices are likely to continue going up due to the avian flu outbreak, with eggs now average $2.88 per dozen, up 52% since the first confirmed case of avian influenza in February."
}}

example 2:
{{
  "Name": "The article's author (unnamed)",
  "Biography": "The author of the article",
  "Information": "The author stated that Wing, which is collaborating with FedEx and Walgreens on drone delivery, was the first to receive a limited Part 135 certificate. Wing is launching operations in Virginia this month, and the Standard certification allows UPS to send an unlimited number of drones to the skies, for their cargo load to exceed 55 pounds and for them to fly at night."
}}

example 3:
{{
   "Name": "Delta's customers",
   "Biography": "People who travel with Delta Air Lines",
   "Information": "Delta's customers suggested that they preferred more space on flights amid the COVID-19 pandemic, and they continue to tell Delta that more space provides more peace of mind."
}}

example 4:
{{
   "Name": "European Union countries",
   "Biography": "Countries that are part of the European Union",
   "Information": "European Union countries are working on adopting copyright rules that allow news companies and publishers to negotiate payments with large tech companies like Facebook, Microsoft and Google that use their content on their platforms."
}}

Output the summary in a list of python dictionaries as in the examples. Don't say anything else.
"""

In [367]:
prompt = SOURCE_EXTRACTION_PROMPT.format(news_article=cleaned_article_text)
extracted_sources = query_together(prompt)

In [368]:
import ast
new_source_df = pd.DataFrame(robust_extract_json_str(extracted_sources))

In [365]:
new_source_df

,Name,Biography,Information
0,Meera Navlakha,The author of the article,"Meera Navlakha stated that Threads, Meta's Twi..."
1,Indiana University,A university that defines accessibility,Indiana University defines accessibility as 't...
2,The CDC,The Centers for Disease Control and Prevention,The CDC reported that roughly one in four adul...
3,Chase DiBenedetto,A writer for Mashable,Chase DiBenedetto wrote that the process of ad...


In [332]:
old_source_df = pd.DataFrame(test_set_df.loc[idx]['sources'])[['Name', 'Information', 'Information', ]]

In [361]:
print(cleaned_article_text)

Threads Needs User-Generated Alt Text ASAP

Of all the missing features, this one is standing out.

by Meera Navlakha 
July 6, 2023

Threads, Meta's Twitter alternative, is here, carrying some Instagram features in tow but really lacking when it comes to others. One of these is alternative text (alt text), a crucial tool for accessibility that tech platforms have been integrating for a while. So far, Threads includes "core accessibility features" that Instagram already has -- it was built by the same team. This means screen-reader support and AI-generated image descriptions are enabled on the new app. But Threads doesn't allow users to add their own text to images, and AI doesn't always do the trick.

Indiana University defines accessibility as "the degree to which a product, device, service, environment, or facility is usable by as many people as possible". In line with this, alt-text descriptions are meant to provide meaning and explanation to content, concisely explaining what each 

In [335]:
# print(new_source_df.to_json(orient='records', lines=True))
new_source_df.to_dict(orient='records')

[{'Name': 'Meera Navlakha',
  'Biography': 'The author of the article',
  'Information': "Meera Navlakha stated that Threads, Meta's Twitter alternative, is lacking in alternative text (alt text), a crucial tool for accessibility that tech platforms have been integrating for a while, and that Threads doesn't allow users to add their own text to images, and AI doesn't always do the trick."},
 {'Name': 'Indiana University',
  'Biography': 'A university that defines accessibility',
  'Information': "Indiana University defines accessibility as 'the degree to which a product, device, service, environment, or facility is usable by as many people as possible', and in line with this, alt-text descriptions are meant to provide meaning and explanation to content, concisely explaining what each is offering."},
 {'Name': 'The CDC',
  'Biography': 'The Centers for Disease Control and Prevention',
  'Information': 'The CDC reported that roughly one in four adults in the United States live with some 

In [350]:
NARRATIVE_KEYWORD_PROMPT = """
You will receive a news article and a set of sources to examine in that article.

For each source in the list, provide the following information, once per source:
    (1) Name: Exactly copy the name of the source.
    (2) Narrative Function: Give a generic keyword label to categorize the narrative role the source playes in the article. 
    Infer why the author used the source, and a generalizable statement about the role they play in the article.
    Don't just summarize their identity. Return in the format: "LABEL": DESCRIPTION.

Here are example outputs. Again, your main task here is to identify a generalizable label that can characterize the narrative role of each source and why the author used them. 

[Examples]
Example 1:

{{
    "Name": "Match Group",
    "Narrative Function": "\"Counterpoint\": This source is used to compare to the main actor in the news article and provide grounding."
}}

Example 2:

{{
    "Name": "Dubai Airshow",
    "Narrative Function": "\"More Context\": This source is used to further expand the context offered and offer a visual setting."
}}

Example 3:
{{

    "Name": "Ann Gough",
    "Narrative Function": "\"Victim\": This source provides the voice of a user for the product, giving us a personal view of the harm caused by the event.
}}

[Instructions]

Now it's your turn. Here is a news article:

```{news_article}```

Please examine the narrative role of each of the following sources: 

```[{target_sources}]```

For each source, answer the questions above. Output the summary in a list of python dictionaries as in the examples. Don't say anything else.
"""

In [351]:
source_list = ', '.join(map(lambda x: '"%s"' % x , new_source_df['Name'].tolist()))

In [352]:
prompt = NARRATIVE_KEYWORD_PROMPT.format(news_article=cleaned_article_text, target_sources=source_list)
narr_function = query_together(prompt)

In [353]:
print(narr_function)

[
    {
        "Name": "Meera Navlakha",
        "Narrative Function": "Authoritative Voice": This source is used as the primary voice of the article, providing the main argument and perspective.
    },
    {
        "Name": "Indiana University",
        "Narrative Function": "Definitional Authority": This source is used to provide a credible definition of a key term, adding depth and context to the article.
    },
    {
        "Name": "The CDC",
        "Narrative Function": "Statistical Evidence": This source is used to provide data and statistics that support the article's argument, adding credibility and weight to the narrative.
    },
    {
        "Name": "Chase DiBenedetto",
        "Narrative Function": "Expert Endorsement": This source is used to provide additional expert opinion and validation of the article's argument, lending credibility to the author's perspective.
    }
]


In [354]:
source_list

'"Meera Navlakha", "Indiana University", "The CDC", "Chase DiBenedetto"'

# Run Ollama

I ran llama3 locally using a cool program called Ollama: https://ollama.com/. You can follow the link to install the program.

This can be useful for trying things out before we work out compute access.

Run this in your terminal:

`ollama run llama3`

Even llama3 8b is pretty powerful. Llama 70b is better, but that may not run on your local computer.

In [ ]:
import requests
import pyperclip

In [ ]:
pyperclip.copy(doc_str)
pyperclip.copy(json_str)

In [ ]:
r = requests.post(
    'http://localhost:11434/api/generate',
    json = {
        "model": "llama3",
        "prompt":f"""
            Here is a news article, with each sentence annotated according to the source of it's information:
            ```
            {json_str}
            ```

            Please summarize each of our source annotations. Tell me in one paragraph per source: (1) who the source is (2) what informational content they provide to the article.
            Only rely on the annotations I have provided, don't identify additional sources.
        """,
        "stream": False
})

# Get articles covering the same press release

In [16]:
a_to_pr_df = pd.read_csv('../../../bloomberg-research/press-releases/data/s_p_500_backlinks/article-to-pr-mapper.csv', index_col=0)

In [20]:
a_to_pr_df['Target URL'].value_counts().value_counts()

count
1      127076
2       21442
3        8404
4        4573
5        2934
        ...  
284         1
282         1
276         1
275         1
177         1
Name: count, Length: 343, dtype: int64

In [73]:
candidate_df = (
    a_to_pr_df
        .loc[lambda df: df['URL'].isin(df['URL'].value_counts().loc[lambda s: s == 1].index)]    
        .loc[lambda df: df['Target URL'].str.len() > 20]    
        .loc[lambda df: df['Target URL'].isin(df['Target URL'].value_counts().loc[lambda s: (s > 2) & (s < 100)].index)]
        # 
        # covered by multiple news outlets
        # 
        # .loc[lambda df: df['Target URL'].isin(
        #     df
        #          [['Target URL', 'news_url_domain']]
        #          .value_counts().unstack()
        #          .fillna(0).pipe(lambda df: (df > 0).astype(int))
        #          .sum(axis=1)
        #          .loc[lambda s: s > 1]
        #          .index
        # )]       
)

In [54]:
a_to_pr_df.shape

(826819, 6)

In [74]:
candidate_df.shape

(107654, 6)

In [55]:
candidate_df.shape

(120993, 6)

In [100]:
candidate_df.to_csv('cache/news_articles_covering_the_same_press_releases.csv')

In [101]:
! open .

In [ ]:
import pandas as pd 
candidate_df = pd.read_csv('cache/news_articles_covering_the_same_press_releases.csv')
pr_url_set = set(candidate_df['Target URL'].drop_duplicates().values)
filtered_pr_d = article_d.filter(lambda x: x['article_url'] in pr_url_set, num_proc=10)

In [89]:
filtered_pr_d

Dataset({
    features: ['article_url', 'target_timestamp_key', 'target_timestamp', 'sort_criteria', 'wayback_url', 'wayback_timestamp', 'method', 'links', 'article_text', 'word_lists', 'sent_lists', 'best_class', 'coref_resolved_sents'],
    num_rows: 9607
})

In [95]:
filtered_pr_df = filtered_pr_d.to_pandas()